#### 1. Import thư viện

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import plotly.graph_objects as go

#### 2. Đọc dữ liệu và chuẩn hoá

In [2]:
df = pd.read_csv('Data/trending_videos_unique.csv')

In [3]:
df['published_at'] = pd.to_datetime(df['published_at'], errors='coerce')
df['snapshot_date'] = pd.to_datetime(df['snapshot_date'], errors='coerce')
df['trending_time_days'] = df['trending_time'].str.replace(
    ' days', '').astype(int)

#### 3. Xây dựng ma trận tương quan

In [4]:
correlation_matrix = df[['view_count', 'like_count',
                         'comment_count', 'trending_time_days']].corr()
correlation_matrix

,view_count,like_count,comment_count,trending_time_days
view_count,1.000000,0.879655,0.288432,0.268101
like_count,0.879655,1.000000,0.366000,0.204704
comment_count,0.288432,0.366000,1.000000,0.118849
trending_time_days,0.268101,0.204704,0.118849,1.000000


#### 4. Chọn những cột cần thiết và cần dự đoán

In [5]:
feature_columns = ['view_count', 'like_count', 'comment_count']
X = df[feature_columns]
y = df['trending_time_days']

#### 5. Chia dữ liệu thành tập huấn luyện và kiểm tra

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f'Số lượng mẫu trong tập huấn luyện: {X_train.shape[0]}')
print(f'Số lượng mẫu trong tập kiểm tra: {X_test.shape[0]}')

Số lượng mẫu trong tập huấn luyện: 1849
Số lượng mẫu trong tập kiểm tra: 463


#### 6. Xây dựng mô hình dự đoán

In [7]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [8]:
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'\nĐánh Giá Mô Hình Linear Regression:')
print(f'Sai số Trung bình Tuyệt đối (MAE): {mae:.2f} ngày')
print(f'Sai số Trung bình Bình phương (MSE): {mse:.2f} ngày^2')
print(f'Căn Bậc Hai của MSE (RMSE): {rmse:.2f} ngày')
print(f'Hệ số Xác định (R² Score): {r2:.2f}')


Đánh Giá Mô Hình Linear Regression:
Sai số Trung bình Tuyệt đối (MAE): 3.49 ngày
Sai số Trung bình Bình phương (MSE): 18.69 ngày^2
Căn Bậc Hai của MSE (RMSE): 4.32 ngày
Hệ số Xác định (R² Score): 0.10


#### 7. Biểu đồ so sánh

In [9]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test, y=y_pred, mode='markers', name='Dự Đoán'))
fig.add_trace(go.Scatter(x=y_test, y=y_test, mode='lines', name='Thực Tế'))
fig.update_layout(title='So sánh Thực Tế và Dự Đoán Thời Gian Trending',
                  xaxis_title='Thời Gian Trending Thực tế (ngày)',
                  yaxis_title='Thời Gian Trending Dự Đoán (ngày)')
fig.show()

#### 8. Ví dụ

In [10]:
def predict_trending_time(view_count, like_count, comment_count):
    new_data = pd.DataFrame([[
        view_count, like_count, comment_count,
    ]],
        columns=feature_columns
    )
    predicted_trending_time = model.predict(new_data)[0]
    return predicted_trending_time


predicted_days = predict_trending_time(
    view_count=1000000,
    like_count=50000,
    comment_count=1000,
)

print(f'Thời gian Trending Dự đoán: {predicted_days:.2f} ngày')

Thời gian Trending Dự đoán: 7.06 ngày
